In [1]:
from confluent_kafka import Consumer, KafkaError
import json
import csv
import os
import time
from datetime import datetime

TOPIC_NAME = "iot_sensors"
BATCH_DIR = "_IOT_Batches"
LOG_FILE = "_IOT_Batches/batch_log.csv"
BATCH_DURATION = 60  
GROUP_ID = "iot-consumer-Grad"  

os.makedirs(BATCH_DIR, exist_ok=True)

consumer_conf = {
    'bootstrap.servers': 'kafka:9092',
    'group.id': GROUP_ID,
    'enable.auto.commit': True,
    'auto.commit.interval.ms': 2000,
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(consumer_conf)
consumer.subscribe([TOPIC_NAME])

print(f" Using new group ID: {GROUP_ID}")
print(f" Listening to topic '{TOPIC_NAME}'...")

existing_batches = []
for f in os.listdir(BATCH_DIR):
    if f.startswith("batch_") and f.endswith(".csv"):
        part = f.split("_")[1].split(".")[0]
        if part.isdigit():
            existing_batches.append(int(part))

batch_number = max(existing_batches, default=0) + 1
def create_new_batch_file():
    global batch_number
    filename = os.path.join(BATCH_DIR, f"batch_{batch_number}.csv")
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'timestamp', 'device_id', 'temperature', 'humidity', 'pressure',
            'air_quality', 'noise_level', 'battery_level', 'signal_strength',
            'latitude', 'longitude', 'raw_message'
        ])
    print(f"\n Started new batch file: {filename}")
    batch_number += 1
    return filename

if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, 'w', newline='', encoding='utf-8') as logf:
        writer = csv.writer(logf)
        writer.writerow(['batch_file', 'start_time', 'end_time', 'message_count'])

csv_file = create_new_batch_file()
batch_start_time = time.time()
message_count = 0
batch_start_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

try:
    while True:
        msg = consumer.poll(1.0)

        if time.time() - batch_start_time >= BATCH_DURATION:
            batch_end_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            with open(LOG_FILE, 'a', newline='', encoding='utf-8') as logf:
                writer = csv.writer(logf)
                writer.writerow([csv_file, batch_start_dt, batch_end_dt, message_count])

            print(f"Batch saved ({message_count} messages): {csv_file}")

            csv_file = create_new_batch_file()
            batch_start_time = time.time()
            message_count = 0
            batch_start_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() != KafkaError._PARTITION_EOF:
                print(f" Error: {msg.error()}")
            continue

        try:
            data = json.loads(msg.value().decode('utf-8'))

            with open(csv_file, 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([
                    data.get('timestamp', ''),
                    data.get('device_id', ''),
                    data.get('temperature', ''),
                    data.get('humidity', ''),
                    data.get('pressure', ''),
                    data.get('air_quality', ''),
                    data.get('noise_level', ''),
                    data.get('battery_level', ''),
                    data.get('signal_strength', ''),
                    data.get('latitude', ''),
                    data.get('longitude', ''),
                    json.dumps(data)
                ])
            message_count += 1
            print(f" Message #{message_count} saved")

        except json.JSONDecodeError:
            print(" Invalid JSON message:", msg.value().decode('utf-8'))
        except Exception as e:
            print(" Error writing message:", e)

except KeyboardInterrupt:
    print("\n Consumer stopped manually.")
finally:
    consumer.close()
    print(" Kafka consumer closed safely.")


 Using new group ID: iot-consumer-Grad
 Listening to topic 'iot_sensors'...

 Started new batch file: _IOT_Batches/batch_17.csv
 Message #1 saved
 Message #2 saved
 Message #3 saved
 Message #4 saved
 Message #5 saved
 Message #6 saved
 Message #7 saved
 Message #8 saved
 Message #9 saved
 Message #10 saved
 Message #11 saved
 Message #12 saved
 Message #13 saved
 Message #14 saved
 Message #15 saved
 Message #16 saved
 Message #17 saved
 Message #18 saved
Batch saved (18 messages): _IOT_Batches/batch_17.csv

 Started new batch file: _IOT_Batches/batch_18.csv

 Consumer stopped manually.
 Kafka consumer closed safely.
